# PSET 3

## Question 1

Consider the de-sparsified Lasso process and the double selection process (described in Section 4.4 of the textbook), which are alternative processes to the double Lasso; in that they are also Neyman orthogonal processes that allow for valid inference in high-dimensional data.


### a

Show that the de-sprasified lasso satisfies the Neyman orthogonality property

## Neyman Orthogonality of the De-sparsified Lasso

To show that the de-sparsified Lasso satisfies the Neyman orthogonality property, we consider the key moment condition involved in the estimation process:

$\[
M(a, \eta) = E[(Y - aD - b'W)\tilde{D}(\gamma)] = 0
\]$

where:
- $\( \eta = (b', \gamma')' \)$ represents the nuisance parameters,
- $\( \tilde{D}(\gamma) = D - \gamma'W \)$ represents the residual from regressing  D onto  W ,
- $\( \gamma \)$ is the best linear predictor coefficient obtained from regressing  D  on  W.

The estimator $ \alpha(\eta) $ is defined as:

$[
\alpha(\eta) = \left(E[D\tilde{D}(\gamma)]\right)^{-1} E[(Y - b'W)\tilde{D}(\gamma)]
]$

For Neyman orthogonality to hold, the moment condition must be insensitive to small perturbations in the nuisance parameters around their true values $ (\eta_0) $. Mathematically, this is expressed as:

\[
\frac{\partial}{\partial \eta} \alpha(\eta) \bigg|_{\eta = \eta_0} = 0
\]

or equivalently:

\[
\frac{\partial}{\partial \eta} M(\alpha(\eta_0), \eta_0) = 0
\]

This implies that the score function is orthogonal to the nuisance parameters at their true values. Since \( \alpha(\eta) \) satisfies this condition, the **de-sparsified Lasso estimator** is **Neyman orthogonal**.

This orthogonality ensures that the estimator is robust to small errors in the estimation of nuisance parameters, which is critical for valid inference in high-dimensional settings.
